In [1]:
import numpy as np
np.random.seed(42)
np.set_printoptions(suppress=True)
from scipy.optimize import least_squares, curve_fit
from matplotlib import pylab as plt
import pandas as pd
import glob
import time
from tqdm.notebook import tqdm
import os
import random

import warnings
from scipy.optimize import OptimizeWarning
warnings.simplefilter("error", OptimizeWarning)

# import seaborn as sns
# sns.set()

In [2]:
def plot_allbands_df(df,title=None,ax=plt):
    ugrizYcolors = ["violet","green","red","darkred","grey","black"]
    for i,band in enumerate("ugrizY"):
        subdf = df[df["band"]==band].reset_index(drop=True)
        x = subdf["mjd"]
        y = subdf["flux"]
        yerr = subdf["fluxerr"]
#         ax.errorbar(x,y,yerr=yerr,ls = "None",color=ugrizYcolors[i])
        ax.scatter(x,y,s=5,label=band,color=ugrizYcolors[i])
        if title is not None:
            if ax is not plt:
                ax.set_title(title)
            else:
                ax.title(title)
    return ax

In [3]:
filels = glob.glob("csv_data/*.csv")

# NEW

In [243]:
file = random.sample(filels,1)[0]
title=file.split("/")[-1]
title_class = title.split("_")[-1].split(".")[0]
print(title)

54015_SNII.csv


In [244]:
df = pd.read_csv(file)

In [245]:
df["mjd"] = df["mjd"] - df["mjd"][0]
df["ratio"] = abs(df["flux"]/df["fluxerr"])

subdf_detected = df[df["detected_bool"]==1]
subdf_ratio2 = df[df["ratio"]>=2]

In [246]:
# tempdf = pd.DataFrame({
#     'brand': ['Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
#     'style': ['cup', 'cup', 'cup', 'pack', 'pack'],
#     'rating': [4, 4, 3.5, 15, 4]
# })

# tempdf.duplicated(keep=False)

In [247]:
smalldf = subdf_ratio2[abs(subdf_ratio2.mjd.diff(periods=-1))<=5]

In [248]:
smalldf2 = smalldf[abs(smalldf.mjd.diff(periods=-1))<=5]

In [249]:
smalldf3 = smalldf2[abs(smalldf2.mjd.diff(periods=-1))<=5]

In [250]:
smalldf4 = smalldf3[abs(smalldf3.mjd.diff(periods=-1))<=5]

In [254]:
subdf_detected.min()

mjd              821.8239
band                    g
flux             2.277386
fluxerr          0.879395
detected_bool           1
ratio            1.125428
dtype: object

In [251]:
subdf_ratio2

,mjd,band,flux,fluxerr,detected_bool,ratio
3,0.0261,z,-6.102359,2.614335,0,2.334192
35,58.8468,z,-7.791263,2.688723,0,2.897756
72,97.8364,z,-5.919583,2.202246,0,2.687975
75,100.8390,g,3.177536,1.405432,0,2.260896
78,100.8686,Y,16.134323,7.940204,0,2.031978
92,115.7397,g,-4.336589,1.978254,0,2.192130
104,128.8131,z,4.841210,2.310705,0,2.095122
140,384.9717,z,6.099794,3.009545,0,2.026816
164,424.9364,i,-4.326866,1.658271,0,2.609264
170,428.0394,z,-7.274901,3.083940,0,2.358963


In [252]:
smalldf4

,mjd,band,flux,fluxerr,detected_bool,ratio
288,815.7598,u,4.928955,1.788882,0,2.755327
290,817.7564,u,4.569284,2.057905,0,2.220357
291,818.7532,u,7.982957,1.630990,0,4.894547
292,821.8239,u,7.228691,1.766931,1,4.091100
293,822.9468,r,8.101382,0.952307,1,8.507112
294,822.9544,g,14.802704,0.979296,1,15.115659
295,822.9620,i,9.990018,1.659150,0,6.021166
296,822.9729,z,6.779741,2.752186,0,2.463402
303,834.7336,r,7.063200,1.099974,1,6.421243
304,834.7412,g,3.465492,1.071537,0,3.234132


In [177]:
abs(subdf_ratio2[abs(subdf_ratio2.mjd.diff(periods=-1))<=5].mjd.diff(periods=-1))<=5

55     False
81     False
142     True
145     True
147    False
251    False
277    False
309     True
310     True
311     True
312     True
313     True
314     True
316     True
317    False
319     True
320     True
321     True
322     True
323     True
324     True
325     True
326     True
327     True
328     True
329     True
330     True
331     True
332    False
341     True
342     True
343     True
344     True
346     True
348     True
349    False
Name: mjd, dtype: bool

In [68]:
subdf_ratio2[["mjd"]].diff(periods=-1).abs()

,mjd
39,22.0194
55,2.9159
58,14.9288
67,12.0323
81,2.9641
82,15.0196
100,241.3972
128,17.7942
133,20.9398
142,0.0300


In [43]:
subdf_ratio2[subdf_ratio2["mjd"].diff()<5]

,mjd,band,flux,fluxerr,detected_bool,ratio
58,68.7275,i,3.212791,1.377818,0,2.331796
82,98.6527,r,-1.715885,0.731345,0,2.346205
145,393.8335,Y,16.688498,7.936971,0,2.102628
147,397.8151,g,-1.558479,0.740076,0,2.105836
149,397.8339,z,-4.030503,1.849723,0,2.178977
254,761.9257,Y,10.545160,4.521957,0,2.331990
278,786.7300,u,-4.174566,1.779315,0,2.346165
310,822.6574,g,4.224734,1.145639,0,3.687666
311,822.6651,i,7.022067,1.348670,0,5.206661
312,822.6763,z,8.685255,1.932950,0,4.493264


In [21]:
mjd_detected_min = subdf_detected.mjd.min()
mjd_detected_max = subdf_detected.mjd.max()

mjd_ratio_min = 

822.6496000000043